# Business Data Analytics - Exercise Deep Learning

In this notebook, we will apply transfer learning on the [CIFAR-10](http://www.cs.toronto.edu/~kriz/cifar.html) dataset. First, we will extract bottleneck features from the pretrained model and then, build our own model on top. The pretrained neural network we will work with is the [Inception-V3](https://arxiv.org/abs/1512.00567v3), but the general idea of this approach can be applied to any other pretrained neural network. The more similar the training data of the pretrained model to the training data of the task at hand, the better features can we extract from the pretrained model. In our case, the Inception-V3 model was trained on the [ImageNet](https://de.wikipedia.org/wiki/ImageNet), which is fairly similar.

### Import libraries

In [ ]:
!pip install -r requirements.txt

In [ ]:
# data wrangling
import pandas as pd
import numpy as np
from PIL import Image

# visualization 
import matplotlib.pyplot as plt
import seaborn as sns

# machine learning and deep learning
from sklearn.metrics import confusion_matrix
import tensorflow as tf
import keras
from keras.datasets import cifar10
from keras.callbacks import ModelCheckpoint   
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, GlobalAveragePooling2D
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.utils import np_utils

### Load and inspect the data

We will first load the data and, to make our lifes easier, limit the dataset size so that training our own neural network becomes feasible.

In [ ]:
# load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data() 

In [ ]:
# limit dataset size if you want to see quick results
x_train, y_train, x_test, y_test = x_train[:2500], y_train[:2500], x_test[:1000], y_test[:1000]

In [ ]:
# inspect final shapes
print('x_train shape:\t', x_train.shape)
print('x_test shape:\t', x_test.shape)

In [ ]:
# Plot an image for each class
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
num_classes = 10

#### Task: Plot an image of each class

Fill out this function to plot an image of each class

In [ ]:
def plot_image_of_each_class():

    # your code goes here
    
plot_image_of_each_class()

### Extract features from pretrained model

In [ ]:
# load model
# it is recommended to use an image shape greater than (75, 75)
# we exclude the top layers so that we access the feature layers directly
model = InceptionV3(weights='imagenet', include_top=False, input_shape=(139, 139, 3))

In [ ]:
# as we have seen earlier, the input shape of our images are not in the required shape. hence, we need to upscale them. 
# as a last step, we need to apply the same preprocessing steps that the images have undergone before they were fed 
# into the inception model for training
def preprocess_raw_image(x: np.ndarray) -> np.ndarray:
    
    # resize images and transform to array
    x_resized = np.array([np.array(Image.fromarray(x[i]).resize((139, 139))) for i in range(0, len(x))]).astype('float32')
    
    # further process data according to requirements of inceptionv3
    x_inception = preprocess_input(x_resized)
    
    return x_inception

In [ ]:
# pre-process data for feature extraction
x_train = preprocess_raw_image(x_train)
x_test = preprocess_raw_image(x_test)

In [ ]:
# check for shape
assert x_train.shape[1:] == (139, 139, 3), 'shape values of training data are not in the required form'
assert x_test.shape[1:] == (139, 139, 3), 'shape values of test data are not in the required form'

#### Task: Extract features from preprocessed images

Fill out the function to obtain the features from our pretrained model. 

In [ ]:
def get_bottle_neck_features(x: np.ndarray) -> np.ndarray:
    
    # your code goes here

In [ ]:
# extract bottleneck features
# this might take a while
features_train = get_bottle_neck_features(x_train)
features_test = get_bottle_neck_features(x_test)

In [ ]:
# one hot encode features
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

### Train own neural network

Now it is time to define and train our very first neural network.

#### Task: Build a simple sequential neural network

As an inspiration, you can have a look at the following architecture:

<img src="cnn.png" align="left"/>

In [ ]:
# define model
def simple_model(image_shape):
    
    # your code goes here

simple_nn = simple_model(image_shape=features_train.shape[1:])

In [ ]:
# inspect model architecture
print(simple_nn.summary())

In [ ]:
# compile model and choose optimizer, loss function and evaluation metric
simple_nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Declare variables
batch_size = 32  # the size the batch size, the more updates during an epoch
epochs = 10  # repeat n times

In [ ]:
# Fit model
history = simple_nn.fit(features_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.25, shuffle=True)

#### Task: Evaluate the learning progess of the model by plotting its loss 

You can use the history variable from above to get the loss and accuracy values.

#### Task: Evaluate your model by calculating the accuracy and confusion matrix

Now it is time to test your model on the test data. Use the accuracy metric to get an overall picture of your model and the confusion matrix to dive deeper into your analysis. To plot the confusion matrix, checkout seaborn for a [heatmap](https://seaborn.pydata.org/generated/seaborn.heatmap.html) that helps to visialize the predictions of our model. The confusion matrix itself and the accuracy can calculated with the help of [sklearn's metric functions](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics).